# 6.2. [**Extraction de caractéristique**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/6_2_feature_extraction.ipynb)</br>([*Feature extraction*](https://scikit-learn.org/stable/modules/feature_extraction.html))

Le module [**`sklearn.feature_extraction`**](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction) peut être utilisé pour extraire des caractéristiques dans un format pris en charge par les algorithmes d'apprentissage automatique à partir de jeux de données consistant en formats tels que le texte et l'image.

**Remarque** : L'extraction de caractéristiques est très différente de la [**Sélection de caractéristiques** (1.13)](https://scikit-learn.org/stable/modules/feature_selection.html#feature-selection) : la première consiste à transformer des données arbitraires, telles que du texte ou des images, en caractéristiques numériques utilisables pour l'apprentissage automatique. La seconde est une technique d'apprentissage automatique appliquée à ces caractéristiques.

# <a id='dict-feature-extraction'></a> 6.2.1. Charger des caractéristiques depuis des dictionnaires

[DictVectorizer]: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html

La classe [**`DictVectorizer`**][DictVectorizer] peut être utilisée pour convertir des tableaux de caractéristiques représentés sous forme de listes d'objets Python standard en la représentation NumPy / SciPy utilisée par les estimateurs scikit-learn. Bien qu'il ne soit pas particulièrement rapide à traiter, le dictionnaire Python a l'avantage d'être pratique à utiliser, d'être creux (les caractéristiques absentes n'ont pas besoin d'être stockées) et de stocker les noms de caractéristiques en plus de leurs valeurs.

[**`DictVectorizer`**][DictVectorizer] implémente ce qu'on appelle le codage "un-sur-K" ou "un-chaud" pour les caractéristiques catégorielles (nominales, discrètes). Les caractéristiques catégorielles sont des paires "attribut-valeur" où la valeur est restreinte à une liste de possibilités discrètes sans ordre (par exemple des identificateurs de sujet, des types d'objets, des balises, des noms...).

Dans ce qui suit, "ville" est un attribut catégoriel tandis que "température" est une caractéristique numérique traditionnelle.

In [1]:
measurements = [
    {'city': 'Dubai', 'temperature': 33.},
    {'city': 'London', 'temperature': 12.},
    {'city': 'San Francisco', 'temperature': 18.},
]

from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer()

vec.fit_transform(measurements).toarray()
# array([[ 1.,  0.,  0., 33.],
#        [ 0.,  1.,  0., 12.],
#        [ 0.,  0.,  1., 18.]])

vec.get_feature_names_out()
# array(['city=Dubai', 'city=London', 'city=San Francisco', 'temperature'], ...)

array(['city=Dubai', 'city=London', 'city=San Francisco', 'temperature'],
      dtype=object)

[**`DictVectorizer`**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html) accepte des valeurs de chaîne multiples pour une caractéristique, par exemple des catégories multiples pour un film.

Supposons qu'une base de données classe chaque film en utilisant certaines catégories (non obligatoires) et son année de sortie.

In [2]:
movie_entry = [
    {'category': ['thriller', 'drama'], 'year': 2003},
    {'category': ['animation', 'family'], 'year': 2011},
    {'year': 1974}
]
vec.fit_transform(movie_entry).toarray()
# array([[0.000e+00, 1.000e+00, 0.000e+00, 1.000e+00, 2.003e+03],
#        [1.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 2.011e+03],
#        [0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.974e+03]])
vec.get_feature_names_out()
# array(['category=animation', 'category=drama', 'category=family',
#        'category=thriller', 'year'], ...)
vec.transform({'category': ['thriller'], 'unseen_feature': '3'}).toarray()
# array([[0., 0., 0., 1., 0.]])

array([[0., 0., 0., 1., 0.]])

[DictVectorizer]: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html

[**`DictVectorizer`**][DictVectorizer] peut également être utilisé pour la transformation de représentation pour entraîner des classificateurs de séquences dans les modèles de traitement automatique de la langue naturelle qui fonctionnent généralement en extrayant des fenêtres de caractéristiques autour d'un mot particulier. Par exemple, supposons qu'on a un premier algorithme qui extrait les étiquettes de partie du discours (PoS) que l'on souhaite utiliser comme étiquettes complémentaires pour entraîner un classificateur de séquences (par exemple un analyseur de segments). Le dictionnaire suivant pourrait être une telle fenêtre de caractéristiques extraites autour du mot "sat" dans la phrase "The cat sat on the mat." :

In [4]:
pos_window = [{
    'word-2': 'the',
    'pos-2': 'DT',
    'word-1': 'cat',
    'pos-1': 'NN',
    'word+1': 'on',
    'pos+1': 'PP',
    },
    # in a real application one would extract many such dictionaries
]

[TfidfTransformer]: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html

La description peut être vectorisée en une matrice creuse bidimensionnelle adaptée pour être utilisée dans un classificateur (peut-être après avoir été utilisée dans un [**`TfidfTransformer`**][TfidfTransformer] pour la normalisation) : 

In [5]:
vec = DictVectorizer()
pos_vectorized = vec.fit_transform(pos_window)
pos_vectorized
# <1x6 sparse matrix of type '<... 'numpy.float64'>'
#     with 6 stored elements in Compressed Sparse ... format>
pos_vectorized.toarray()
# array([[1., 1., 1., 1., 1., 1.]])
vec.get_feature_names_out()
# array(['pos+1=PP', 'pos-1=NN', 'pos-2=DT', 'word+1=on', 'word-1=cat',
#        'word-2=the'], ...)

array(['pos+1=PP', 'pos-1=NN', 'pos-2=DT', 'word+1=on', 'word-1=cat',
       'word-2=the'], dtype=object)

Comme vous pouvez l'imaginer, si l'on extrait un tel contexte autour de chaque mot individuel d'un corpus de documents, la matrice résultante sera très large (de nombreuses caractéristiques uniques) avec la plupart d'entre elles valant zéro la plupart du temps. Afin de rendre la structure de données résultante capable de tenir en mémoire, la classe `DictVectorizer` utilise par défaut une matrice `scipy.sparse` plutôt qu'une `numpy.ndarray`.

# <a id='feature-hashing'></a> 6.2.2. Hachage de caractéristique

[FeatureHasher]: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.FeatureHasher.html
[MultinomialNB]: https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html
[chi2]: https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.chi2.html
[CountVectorizer]: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

La classe [**`FeatureHasher`**][FeatureHasher] est un vectoriseur à haute vitesse et faible consommation de mémoire qui utilise une technique connue sous le nom de "hashing trick" ou ["hachage de caractéristiques"](https://en.wikipedia.org/wiki/Feature_hashing). Au lieu de construire une table de hachage des caractéristiques rencontrées lors de l'entraînement, comme le font les vectoriseurs, les instances de [**`FeatureHasher`**][FeatureHasher] appliquent une fonction de hachage aux caractéristiques pour déterminer leur index de colonne dans les matrices d'échantillons directement. Le résultat est une augmentation de la vitesse et une réduction de l'utilisation de la mémoire, aux dépens de l'inspectabilité; le hasher ne se souvient pas de la forme des caractéristiques d'entrée et n'a pas de méthode `inverse_transform`.

Comme la fonction de hachage peut causer des collisions entre des caractéristiques (non liées), une fonction de hachage signée est utilisée et le signe de la valeur de hachage détermine le signe de la valeur stockée dans la matrice de sortie pour une caractéristique. De cette façon, les collisions sont susceptibles d'annuler plutôt que d'accumuler des erreurs, et la moyenne attendue de la valeur de toute caractéristique de sortie est de zéro. Ce mécanisme est activé par défaut avec `alternate_sign=True` et est particulièrement utile pour les petites tailles de table de hachage (`n_features < 10000`). Pour les grandes tailles de table de hachage, il peut être désactivé, pour permettre de passer la sortie à des estimateurs tels que [**`MultinomialNB`**][MultinomialNB] ou des sélecteurs de caractéristiques [**`chi2`**][chi2] qui attendent des entrées non négatives.

[**`FeatureHasher`**][FeatureHasher] accepte soit des mappings (comme les `dict` de Python et ses variantes dans le module `collections`), des paires `(feature, value)` ou des chaînes, en fonction du paramètre `input_type` du constructeur. Les mappings sont traités comme des listes de paires `(feature, value)`, tandis que les chaînes simples ont une valeur implicite de 1, donc ['feat1', 'feat2', 'feat3'] est interprété comme `[('feat1', 1), ('feat2', 1), ('feat3', 1)]`. Si une seule caractéristique apparaît plusieurs fois dans un échantillon, les valeurs associées seront additionnées (donc `('feat', 2)` et `('feat', 3.5)` deviennent `('feat', 5.5)`). La sortie de [**`FeatureHasher`**][FeatureHasher] est toujours une matrice `scipy.sparse` dans le format CSR.

Le hachage de caractéristiques peut être utilisé pour la classification de documents, mais contrairement à [**`CountVectorizer`**][CountVectorizer], [**`FeatureHasher`**][FeatureHasher] ne fait pas de découpage de mots ni aucun autre prétraitement, sauf l'encodage Unicode-to-UTF-8. Voir [**Vectorisation d'un grand corpus de texte à l'aide de l'astuce du hachage** (6.2.3.8)](https://scikit-learn.org/stable/modules/feature_extraction.html#hashing-vectorizer) ci-dessous pour un tokenizer / hasher combiné.

En exemple, considérons une tâche de traitement par mots du langage naturel qui a besoin de caractéristiques extraites à partir de paires `(token, part_of_speech)`. On pourrait utiliser une fonction génératrice de Python pour extraire des caractéristiques :

In [ ]:
def token_features(token, part_of_speech):
    if token.isdigit():
        yield "numeric"
    else:
        yield "token={}".format(token.lower())
        yield "token,pos={},{}".format(token, part_of_speech)
    if token[0].isupper():
        yield "uppercase_initial"
    if token.isupper():
        yield "all_uppercase"
    yield "pos={}".format(part_of_speech)

Ensuite, le `raw_X` à utiliser pour `FeatureHasher.transform` peut être construit en utilisant:

In [ ]:
raw_X = (token_features(tok, pos_tagger(tok)) for tok in corpus)

et donné à un hachage avec:

In [ ]:
hasher = FeatureHasher(input_type='string')
X = hasher.transform(raw_X)

pour obtenir une matrice `scipy.sparse` `X`.

Notez l'utilisation d'une compréhension de générateur, qui introduit de la paresse dans l'extraction de caractéristiques: les jetons ne sont traités que sur demande du hachage.

## <a id='implementation-details'></a> 6.2.2.1. Détails d'implémentation

[FeatureHasher]: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.FeatureHasher.html

[**`FeatureHasher`**][FeatureHasher] utilise la variante signée 32 bits de MurmurHash3. En conséquence (et à cause des limitations de `scipy.sparse`), le nombre maximum de caractéristiques prises en charge est actuellement de $2^{31} - 1$.

La formulation d'origine de l'astuce du hachage par Weinberger et al. utilisait deux fonctions distinctes $h$ et $\xi$ pour déterminer respectivement l'index de colonne et le signe d'une caractéristique. La présente implémentation fonctionne en supposant que le bit de signe de MurmurHash3 est indépendant de ses autres bits.

Comme un simple modulo est utilisé pour transformer la fonction de hachage en un index de colonne, il est conseillé d'utiliser une puissance de deux comme paramètre `n_features`; sinon, les caractéristiques ne seront pas réparties de manière équilibrée sur les colonnes.

## Références

Kilian Weinberger, Anirban Dasgupta, John Langford, Alex Smola and Josh Attenberg (2009). [“**Feature hashing for large scale multitask learning**](https://alex.smola.org/papers/2009/Weinbergeretal09.pdf)[”](https://drive.google.com/file/d/1zcMDHxm0Nf3gCmBpViDg_pQ_5N6GB11I/view?usp=share_link) Proc. ICML.

[**MurmurHash3**](https://github.com/aappleby/smhasher).


# <a id='text-feature-extraction'></a> 6.2.3. Extraction de caractéristiques de texte

## <a id='the-bag-of-words-representation'></a> 6.2.3.1. La représentation sac de mots

L'analyse de texte est un domaine d'application majeur pour les algorithmes d'apprentissage automatique. Cependant, les données brutes, une séquence de symboles, ne peuvent pas être directement données aux algorithmes eux-mêmes car la plupart d'entre eux attendent des vecteurs de caractéristiques numériques de taille fixe plutôt que des documents de texte brut de longueur variable.

Afin de résoudre ce problème, scikit-learn fournit des utilitaires pour les méthodes les plus courantes pour extraire des caractéristiques numériques à partir de contenus de texte, à savoir:
* **Tokenisation** des chaînes et donner un identifiant entier pour chaque jeton possible, par exemple en utilisant des espaces blancs et des signes de ponctuation comme séparateurs de jetons.
* **Comptage** des occurrences de jetons dans chaque document.
* **Normalisation** et **pondération** avec une importance décroissante les jetons qui apparaissent dans la plupart des échantillons/documents.

Dans ce schéma, les caractéristiques et les échantillons sont définis comme suit:
* Chaque **fréquence d'occurrence individuelle de jeton** (normalisée ou non) est traitée comme une **caractéristique**.
* Le vecteur de toutes les fréquences de jetons pour un **document** donné est considéré comme un échantillon **multivarié**.

Un corpus de documents peut ainsi être représenté par une matrice avec une ligne par document et une colonne par jeton (par exemple mot) qui apparaît dans le corpus.

Nous appelons **vectorisation** le processus général de transformer une collection de documents de texte en vecteurs de caractéristiques numériques. Cette stratégie spécifique (tokenisation, comptage et normalisation) est appelée la représentation **sac de mots** ou "sac de n-grammes". Les documents sont décrits par des occurrences de mots en ignorant complètement les informations de position relatives des mots dans le document.

## <a id='sparsity'></a> 6.2.3.2. Sparsity (Rareté)

Comme la plupart des documents utilisent généralement un très petit sous-ensemble des mots utilisés dans le corpus, la matrice résultante aura de nombreuses valeurs de caractéristiques qui sont nulles (généralement plus de 99% d'entre elles).

Par exemple, une collection de 10 000 documents de texte courts (comme des e-mails) utilisera un vocabulaire d'une taille de l'ordre de 100 000 mots uniques au total, tandis que chaque document utilisera individuellement de 100 à 1000 mots uniques.

Afin de pouvoir stocker une telle matrice en mémoire mais également pour accélérer les opérations algébriques matrice / vecteur, les implémentations utiliseront généralement une représentation creuse telle que les implémentations disponibles dans le package `scipy.sparse`.

## <a id='common-vectorizer-usage'></a> 6.2.3.3. Utilisation courante du Vectoriseur

[**`CountVectorizer`**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) implémente à la fois la tokenisation et le décompte des occurrences dans une seule classe :

In [1]:
from sklearn.feature_extraction.text import CountVectorizer

Ce modèle a de nombreux paramètres, mais les valeurs par défaut sont assez raisonnables (veuillez consulter la documentation de référence pour plus de détails) :

In [2]:
vectorizer = CountVectorizer()
vectorizer

CountVectorizer()

Utilisons-le pour tokeniser et compter les occurrences de mots dans un corpus minimaliste de documents textuels :

In [3]:
corpus = [
    'This is the first document.',
    'This is the second document.',
    'And the third one.',
    'Is this the first document?',
]
X = vectorizer.fit_transform(corpus)
X
# <4x9 sparse matrix of type '<class 'numpy.int64'>'
#         with 19 stored elements in Compressed Sparse Row format>

<4x9 sparse matrix of type '<class 'numpy.int64'>'
	with 19 stored elements in Compressed Sparse Row format>

La configuration par défaut tokenise la chaîne en extrayant des mots d'au moins 2 lettres. La fonction spécifique qui réalise cette étape peut être demandée explicitement :

In [4]:
analyze = vectorizer.build_analyzer()
analyze("This is a text document to analyze.") == (
    ['this', 'is', 'text', 'document', 'to', 'analyze']
)
# True

True

Chaque terme trouvé par l'analyseur pendant l'ajustement est affecté à un indice d'entier unique correspondant à une colonne dans la matrice résultante. Cette interprétation des colonnes peut être récupérée comme suit :

In [5]:
vectorizer.get_feature_names_out()
# array(['and', 'document', 'first', 'is', 'one', 'second', 'the',
#        'third', 'this'], ...)

X.toarray()
# array([[0, 1, 1, 1, 0, 0, 1, 0, 1],
#        [0, 1, 0, 1, 0, 2, 1, 0, 1],
#        [1, 0, 0, 0, 1, 0, 1, 1, 0],
#        [0, 1, 1, 1, 0, 0, 1, 0, 1]]...)

array([[0, 1, 1, 1, 0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 2, 1, 0, 1],
       [1, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 1, 1, 1, 0, 0, 1, 0, 1]], dtype=int64)

La correspondance inverse du nom de caractéristique à l'index de colonne est stockée dans l'attribut `vocabulary_` du vectoriseur :

In [6]:
vectorizer.vocabulary_.get('document')
# 1

1

Ainsi, les mots qui n'ont pas été vus dans le corpus d'entraînement seront complètement ignorés lors de futurs appels à la méthode de transformation :

In [7]:
vectorizer.transform(['Something completely new.']).toarray()
# array([[0, 0, 0, 0, 0, 0, 0, 0, 0]]...)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

Notez que dans le corpus précédent, les premiers et derniers documents ont exactement les mêmes mots, ils sont donc encodés en vecteurs égaux. En particulier, nous perdons l'information que le dernier document est une forme interrogative. Pour préserver certaines des informations d'ordre local, nous pouvons extraire des 2-grammes de mots en plus des 1-grammes (mots individuels) :

In [8]:
bigram_vectorizer = CountVectorizer(
    ngram_range=(1, 2),
    token_pattern=r'\b\w+\b',
    min_df=1
)
analyze = bigram_vectorizer.build_analyzer()
analyze('Bi-grams are cool!') == (
    ['bi', 'grams', 'are', 'cool', 'bi grams', 'grams are', 'are cool']
)
# True

True

Le vocabulaire extrait par ce vectoriseur est donc beaucoup plus grand et peut maintenant résoudre les ambiguïtés encodées dans les patrons de positionnement local :

In [9]:
X_2 = bigram_vectorizer.fit_transform(corpus).toarray()
X_2
# array([[0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0],
#        [0, 0, 1, 0, 0, 1, 1, 0, 0, 2, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0],
#        [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0],
#        [0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1]]...)

array([[0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0],
       [0, 0, 1, 0, 0, 1, 1, 0, 0, 2, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1]],
      dtype=int64)

En particulier, la forme interrogative "Est-ce que" n'est présente que dans le dernier document :

In [10]:
feature_index = bigram_vectorizer.vocabulary_.get('is this')
X_2[:, feature_index]
# array([0, 0, 0, 1]...)

array([0, 0, 0, 1], dtype=int64)

### <a id='using-stop-words'></a> Utilisation des mots vides

Les mots vides sont des mots tels que "et", "the" ou "him" qui sont censés être peu informatifs pour représenter le contenu d'un texte et peuvent être enlevés pour éviter d'être considérés comme un signal pour la prédiction. Cependant, parfois, des mots similaires sont utiles pour la prédiction, tels que pour classer le style d'écriture ou la personnalité.

Il y a plusieurs problèmes connus dans notre liste de mots vides "anglais" fournie. Elle n'a pas pour objectif d'être une solution générale "tout en un", car certaines tâches peuvent nécessiter une solution plus personnalisée. Voir [NQY18] pour plus de détails.

Veuillez faire attention au choix d'une liste de mots vides. Les listes populaires de mots vides peuvent inclure des mots très informatifs pour certaines tâches, tels que *computer*.

Vous devez également vous assurer que la liste de mots vides a subi le même prétraitement et la même tokenisation que celui utilisé dans le vectoriseur. Par exemple, le mot *we've* est divisé en *we* et *ve* par le tokeniseur par défaut de `CountVectorizer`, donc si *we've* est dans `stop_words` mais que *ve* ne l'est pas, *ve* sera conservé à partir de *we've* dans le texte transformé. Nos vectoriseurs essayeront d'identifier et d'avertir à propos de certaines incohérences.

#### Références

[NQY18] J. Nothman, H. Qin and R. Yurchak (2018). [“**Stop Word Lists in Free Open-source Software Packages**](https://scikit-learn.org/stable/modules/feature_extraction.html)[”](https://drive.google.com/file/d/1k0wR1OX6ZjWEvmQTdYbFvpKr2eOwE0Dv/view?usp=share_link). In Proc. Workshop for NLP Open Source Software.


## <a id='tfidf-term-weighting'></a> 6.2.3.4. Pondération Tf-idf

[TfidfTransformer]: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html
[TfidfVectorizer]: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

Dans un corpus de texte important, certaines mots seront très présents (par exemple "the", "a", "is" en anglais) et porteront donc très peu d'information significative sur le contenu réel du document. Si nous devions alimenter les données de comptage directement à un classificateur, ces termes très fréquents masqueraient les fréquences de termes plus rares mais plus intéressants.

Afin de rééquilibrer les données de comptage en valeurs flottantes appropriées pour une utilisation par un classificateur, il est très courant d'utiliser la transformation tf-idf.

Tf signifie **fréquence de terme** tandis que tf-idf signifie fréquence de terme multipliée par la **fréquence inverse de document** : $\text{tf-idf(t,d)}=\text{tf(t,d)} \times \text{idf(t)}$.

En utilisant les paramètres par défaut de `TfidfTransformer`, `TfidfTransformer(norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False)`, la fréquence de terme, le nombre de fois qu'un terme apparaît dans un document donné, est multiplié par la composante idf, qui est calculée comme

$\text{idf}(t) = \log{\frac{1 + n}{1+\text{df}(t)}} + 1$, 

où $n$ est le nombre total de documents dans le jeu de documents, et $\text{df}(t)$ est le nombre de documents dans le jeu de documents qui contiennent le terme $t$. Les vecteurs tf-idf résultants sont ensuite normalisés par la norme euclidienne :

$v_{norm} = \frac{v}{||v||_2} = \frac{v}{\sqrt{v{_1}^2 +
v{_2}^2 + \dots + v{_n}^2}}$.

Ceci était à l'origine un schéma de pondération de termes développé pour la recherche d'informations (en tant que fonction de classement pour les résultats de moteur de recherche) qui a également trouvé une bonne utilisation en classification de documents et en clustering.

Les sections suivantes contiennent des explications et des exemples supplémentaires qui illustrent comment les tf-idf sont exactement calculés et comment les tf-idf calculés dans [**`TfidfTransformer`**][TfidfTransformer] et [**`TfidfVectorizer`**][TfidfVectorizer] de scikit-learn diffèrent légèrement de la notation de livre de texte standard qui définit l'idf comme

$\text{idf}(t) = \log{\frac{n}{1+\text{df}(t)}}$

Dans [**`TfidfTransformer`**][TfidfTransformer] et [**`TfidfVectorizer`**][TfidfVectorizer] avec `smooth_idf=False`, le compte "1" est ajouté à l'idf au lieu du dénominateur de l'idf :

$\text{idf}(t) = \log{\frac{n}{\text{df}(t)}} + 1$

Cette normalisation est implémentée par la classe [**`TfidfTransformer`**][TfidfTransformer].

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)
transformer
# TfidfTransformer(smooth_idf=False)

TfidfTransformer(smooth_idf=False)

Encore une fois, veuillez consulter la documentation de référence pour les détails sur tous les paramètres.

Prenons un exemple avec les comptes suivants. Le premier terme est présent à 100% du temps, et n'est donc pas très intéressant. Les deux autres caractéristiques sont présentes moins de 50% du temps, donc probablement plus représentatives du contenu des documents.

In [12]:
counts = [
    [3, 0, 1],
    [2, 0, 0],
    [3, 0, 0],
    [4, 0, 0],
    [3, 2, 0],
    [3, 0, 2]
]
tfidf = transformer.fit_transform(counts)
print(tfidf)
# <6x3 sparse matrix of type '<... 'numpy.float64'>'
#     with 9 stored elements in Compressed Sparse ... format>

tfidf.toarray()
# array([[0.81940995, 0.        , 0.57320793],
#        [1.        , 0.        , 0.        ],
#        [1.        , 0.        , 0.        ],
#        [1.        , 0.        , 0.        ],
#        [0.47330339, 0.88089948, 0.        ],
#        [0.58149261, 0.        , 0.81355169]])

  (0, 2)	0.5732079309279059
  (0, 0)	0.8194099510753754
  (1, 0)	1.0
  (2, 0)	1.0
  (3, 0)	1.0
  (4, 1)	0.8808994832762984
  (4, 0)	0.47330339145578754
  (5, 2)	0.8135516873095774
  (5, 0)	0.5814926070688599


array([[0.81940995, 0.        , 0.57320793],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.47330339, 0.88089948, 0.        ],
       [0.58149261, 0.        , 0.81355169]])

Chaque ligne est normalisée pour avoir une norme euclidienne unitaire:

$$
v_{norm} = \frac{v}{||v||_2} = \frac{v}{\sqrt{v{_1}^2 +
v{_2}^2 + \dots + v{_n}^2}}
$$

Par exemple, nous pouvons calculer le tf-idf du premier terme dans le premier document dans le tableau des effectifs `counts` de la manière suivante:
* $n = 6$
* $\text{df}(t)_{\text{term1}}$
* $\text{idf}(t)_{\text{term1}} = \log \frac{n}{\text{df}(t)} + 1 = \log(1)+1 = 1$
* $\text{tf-idf}_{\text{term1}} = \text{tf} \times \text{idf} = 3 \times 1 = 3$

Maintenant, si nous répétons ce calcul pour les deux autres termes dans le document, nous obtenons
* $\text{tf-idf}_{\text{term2}} = 0 \times (\log(6/1)+1) = 0$
* $\text{tf-idf}_{\text{term3}} = 1 \times (\log(6/2)+1) \approx 2.0986$

et le vecteur de tf-idf bruts:
* $\text{tf-idf}_{\text{raw}} = [3, 0, 2.0986]$

Ensuite, en appliquant la norme euclidienne (L2), nous obtenons les tf-idf suivants pour le document 1:
* $\frac{[3, 0, 2.0986]}{\sqrt{\big(3^2 + 0^2 + 2.0986^2\big)}} = [ 0.819,  0,  0.573].$

De plus, le paramètre par défaut `smooth_idf=True` ajoute "1" au numérateur et au dénominateur, comme si un document supplémentaire avait été vu contenant tous les termes dans la collection exactement une fois, ce qui empêche les divisions par zéro:

$\text{idf}(t) = \log{\frac{1 + n}{1+\text{df}(t)}} + 1$

En utilisant cette modification, le tf-idf du troisième terme dans le document 1 change pour 1,8473:

$\text{tf-idf}_{\text{term3}} = 1 \times \log(7/3)+1 \approx 1.8473$

Et le tf-idf normalisé en L2 change pour:

$\frac{[3, 0, 1.8473]}{\sqrt{\big(3^2 + 0^2 + 1.8473^2\big)}} = [0.8515, 0, 0.5243]$

In [13]:
transformer = TfidfTransformer()
transformer.fit_transform(counts).toarray()
# array([[0.85151335, 0.        , 0.52433293],
#        [1.        , 0.        , 0.        ],
#        [1.        , 0.        , 0.        ],
#        [1.        , 0.        , 0.        ],
#        [0.55422893, 0.83236428, 0.        ],
#        [0.63035731, 0.        , 0.77630514]])

array([[0.85151335, 0.        , 0.52433293],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.55422893, 0.83236428, 0.        ],
       [0.63035731, 0.        , 0.77630514]])

Les poids de chaque caractéristique calculés par l'appel de la méthode `fit` sont stockés dans un attribut du modèle :

In [14]:
transformer.idf_
# array([1. ..., 2.25..., 1.84...])

array([1.        , 2.25276297, 1.84729786])

Comme tf-idf est très souvent utilisé pour les caractéristiques de texte, il existe également une autre classe appelée [**`TfidfVectorizer`**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) qui combine toutes les options de [**`CountVectorizer`**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) et [**`TfidfTransformer`**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html) en un seul modèle :

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit_transform(corpus)
# <4x9 sparse matrix of type '<... 'numpy.float64'>'
#     with 19 stored elements in Compressed Sparse ... format>

Bien que la normalisation tf-idf soit souvent très utile, il peut y avoir des cas où les marqueurs d'occurrence binaire peuvent offrir de meilleures caractéristiques. Cela peut être obtenu en utilisant le paramètre `binary` de [**`CountVectorizer`**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). En particulier, certains estimateurs tels que [**Bernoulli Naive Bayes** (1.9.4)](https://scikit-learn.org/stable/modules/naive_bayes.html#bernoulli-naive-bayes) modèlent explicitement des variables aléatoires booléennes discrètes. De plus, les textes très courts ont tendance à avoir des valeurs tf-idf bruyantes, tandis que les informations d'occurrence binaire sont plus stables.

Comme d'habitude, la meilleure façon d'ajuster les paramètres d'extraction de caractéristiques est d'utiliser une grille de validation croisée, par exemple en installant le extracteur de caractéristiques avec un classificateur :

### Exemples

#### [**Exemple de pipeline pour l'extraction et l'évaluation des caractéristiques de texte**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_grid_search_text_feature_extraction.ipynb)<br/>([*Sample pipeline for text feature extraction and evaluation*](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_text_feature_extraction.html))

## <a id='decoding-text-files'></a> 6.2.3.5. Décodage de fichiers de texte

Le texte est composé de caractères, mais les fichiers sont composés d'octets. Ces octets représentent des caractères selon certaines encodages. Pour travailler avec des fichiers de texte en Python, leurs octets doivent être décodés en un jeu de caractères appelé Unicode. Les encodages courants sont ASCII, Latin-1 (Europe de l'Ouest), KOI8-R (russe) et les encodages universels UTF-8 et UTF-16. Beaucoup d'autres existent.

📌 Il est important de noter que le terme "jeu de caractères" peut aussi être utilisé pour décrire un encodage, mais ce terme est moins précis, car plusieurs encodages peuvent exister pour un seul jeu de caractères.

Les extracteurs de caractéristiques de texte dans scikit-learn sont capables de décoder les fichiers de texte, à condition que l'encodage soit indiqué. Le [**`CountVectorizer`**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) prend un paramètre `encoding` à cet effet. Pour les fichiers de texte modernes, l'encodage par défaut est probablement UTF-8 (`encoding= "utf-8"`).

Si le texte que vous chargez n'est pas réellement encodé en UTF-8, vous obtiendrez une `UnicodeDecodeError`. Vous pouvez éviter ces erreurs en définissant le paramètre `decode_error` soit sur "ignore" ou "replace". Pour plus de détails, veuillez consulter la documentation de la fonction Python `bytes.decode`. Pour plus de détails (tapez `help(bytes.decode)` à l'invite Python).

Si vous rencontrez des difficultés pour décoder du texte, voici quelques solutions à essayer :
* Trouvez l'encodage réel du texte. Le fichier peut comporter un en-tête ou un fichier README qui indique l'encodage, ou vous pouvez supposer un encodage standard en fonction de la source du texte.
* Vous pouvez déterminer le type d'encodage général en utilisant la commande UNIX `file`. Le module Python `chardet` inclut un script appelé `chardetect.py` qui peut deviner l'encodage spécifique, même si vous ne pouvez pas être sûr de la précision de son hypothèse.
* Vous pouvez essayer d'utiliser UTF-8 et ignorer les erreurs. Vous pouvez décoder les chaînes de caractères en octets avec `bytes.decode(errors='replace')` pour remplacer toutes les erreurs de décodage avec un caractère sans signification, ou définir `decode_error='replace'` dans le vectoriseur. Cela peut endommager la pertinence de vos caractéristiques.
* Le texte réel peut provenir de différentes sources qui peuvent avoir utilisé différents encodages ou même être décodé de manière négligente dans un encodage différent de celui avec lequel il a été encodé. C'est courant pour le texte récupéré sur le Web. Le paquet Python [**ftfy**](https://github.com/rspeer/python-ftfy) peut automatiquement trier certains types d'erreurs de décodage, il est donc  possible de décoder le texte inconnu en `latin-1` et de le corriger ensuite à l'aide de `ftfy`.
* Si le texte est dans un mélange d'encodages qui est simplement trop difficile à trier (ce qui est le cas pour le jeu de données 20 Newsgroups), vous pouvez vous rabattre sur un encodage simple à un seul octet tel que `latin-1`. Certaines parties du texte peuvent s'afficher incorrectement, mais au moins la même séquence d'octets représentera toujours la même caractéristique.

Par exemple, l'extrait de code suivant utilise `chardet` (non fourni avec scikit-learn, il doit être installé séparément) pour déterminer l'encodage de trois textes. Il vectorise ensuite les textes et affiche le vocabulaire appris. La sortie n'est pas affichée ici.

In [4]:
import chardet
from sklearn.feature_extraction.text import CountVectorizer
text1 = b"Sei mir gegr\xc3\xbc\xc3\x9ft mein Sauerkraut"
text2 = b"holdselig sind deine Ger\xfcche"
text3 = b"\xff\xfeA\x00u\x00f\x00 \x00F\x00l\x00\xfc\x00g\x00e\x00l\x00n\x00 \x00d\x00e\x00s\x00 \x00G\x00e\x00s\x00a\x00n\x00g\x00e\x00s\x00,\x00 \x00H\x00e\x00r\x00z\x00l\x00i\x00e\x00b\x00c\x00h\x00e\x00n\x00,\x00 \x00t\x00r\x00a\x00g\x00 \x00i\x00c\x00h\x00 \x00d\x00i\x00c\x00h\x00 \x00f\x00o\x00r\x00t\x00"
decoded = [
    x.decode(chardet.detect(x)['encoding'])
    for x in (text1, text2, text3)
]        
v = CountVectorizer().fit(decoded).vocabulary_    
for term in v:
    print(v)

{'sei': 15, 'mir': 13, 'gegrüßt': 6, 'mein': 12, 'sauerkraut': 14, 'holdselig': 10, 'sind': 16, 'deine': 1, 'gerüche': 7, 'auf': 0, 'flügeln': 4, 'des': 2, 'gesanges': 8, 'herzliebchen': 9, 'trag': 17, 'ich': 11, 'dich': 3, 'fort': 5}
{'sei': 15, 'mir': 13, 'gegrüßt': 6, 'mein': 12, 'sauerkraut': 14, 'holdselig': 10, 'sind': 16, 'deine': 1, 'gerüche': 7, 'auf': 0, 'flügeln': 4, 'des': 2, 'gesanges': 8, 'herzliebchen': 9, 'trag': 17, 'ich': 11, 'dich': 3, 'fort': 5}
{'sei': 15, 'mir': 13, 'gegrüßt': 6, 'mein': 12, 'sauerkraut': 14, 'holdselig': 10, 'sind': 16, 'deine': 1, 'gerüche': 7, 'auf': 0, 'flügeln': 4, 'des': 2, 'gesanges': 8, 'herzliebchen': 9, 'trag': 17, 'ich': 11, 'dich': 3, 'fort': 5}
{'sei': 15, 'mir': 13, 'gegrüßt': 6, 'mein': 12, 'sauerkraut': 14, 'holdselig': 10, 'sind': 16, 'deine': 1, 'gerüche': 7, 'auf': 0, 'flügeln': 4, 'des': 2, 'gesanges': 8, 'herzliebchen': 9, 'trag': 17, 'ich': 11, 'dich': 3, 'fort': 5}
{'sei': 15, 'mir': 13, 'gegrüßt': 6, 'mein': 12, 'sauerkraut

(Selon la version de `chardet`, il pourrait se tromper sur le premier.)

Pour une introduction à Unicode et aux encodages de caractères en général, voir l'article [**Le minimum absolu que tout développeur logiciel devrait connaître à propos d'Unicode**](https://www.joelonsoftware.com/2003/10/08/the-absolute-minimum-every-software-developer-absolutely-positively-must-know-about-unicode-and-character-sets-no-excuses/)  de Joel Spolsky.

## <a id='applications-and-examples'></a> 6.2.3.6. Applications et exemples

La représentation "sac de mots" est très simpliste mais surprenamment utile en pratique.

En particulier dans un **environnement supervisé**, elle peut être combinée avec des modèles linéaires rapides et évolutifs pour former des **classificateurs de documents**, par exemple:

* [**Classification de documents textuels à l'aide de caractéristiques rares**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/text/plot_document_classification_20newsgroups.ipynb)<br/>([*Classification of text documents using sparse features*](https://scikit-learn.org/stable/auto_examples/text/plot_document_classification_20newsgroups.html))

Dans un **environnement non supervisé**, elle peut être utilisée pour regrouper des documents similaires en utilisant des algorithmes de clustering tels que K-means:

* [**Clustering de documents textuels à l'aide de k-means**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/text/plot_document_clustering.ipynb)<br/>([*Clustering text documents using k-means*](https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html))

Enfin, il est possible de découvrir les principaux sujets d'un corpus en relaxant la contrainte d'affectation dure du clustering, par exemple en utilisant [**la factorisation de matrice non-négative (NMF ou NNMF) (2.5.7)**](https://scikit-learn.org/stable/modules/decomposition.html#nmf):

* [**Extraction de thème avec la factorisation de matrice non-négative et l'allocation latente de Dirichlet**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/applications/feature_selection/plot_topics_extraction_with_nmf_lda.ipynb)<br/>([*Topic extraction with Non-negative Matrix Factorization and Latent Dirichlet Allocation*](https://scikit-learn.org/stable/applications/feature_selection/plot_topics_extraction_with_nmf_lda.html))

## <a id='limitations-of-the-bag-of-words-representation'></a> 6.2.3.7. Limitations de la représentation Sac de mots

Une collection d'unigrammes (ce que représente le sac de mots) ne peut pas capturer les phrases et les expressions à plusieurs mots, en ignorant en pratique toute dépendance à l'ordre des mots. De plus, le modèle de sac de mots ne prend pas en compte les fautes d'orthographe potentielles ou les dérivations de mots.

Les N-grammes viennent à la rescousse! Au lieu de construire une simple collection d'unigrammes (n = 1), on peut préférer une collection de bigrammes (n = 2), où les occurrences de paires de mots consécutifs sont comptées.

On peut également considérer une collection de n-grammes de caractères, une représentation résiliente aux fautes d'orthographe et aux dérivations.

Par exemple, disons que nous avons affaire à un corpus de deux documents: `['words', 'wprds']`. Le deuxième document contient une faute d'orthographe du mot 'words'. Une simple représentation de sac de mots considérerait ces deux documents comme très distincts, différant dans les deux caractéristiques possibles. Une représentation de 2-grammes de caractères, cependant, trouverait que les documents correspondent dans 4 des 8 caractéristiques, ce qui peut aider le classifieur préféré à décider mieux:

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
ngram_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(2, 2))
counts = ngram_vectorizer.fit_transform(['words', 'wprds'])
ngram_vectorizer.get_feature_names_out()
# array([' w', 'ds', 'or', 'pr', 'rd', 's ', 'wo', 'wp'], ...)
counts.toarray().astype(int)
# array([[1, 1, 1, 0, 1, 1, 1, 0],
#        [1, 1, 0, 1, 1, 1, 0, 1]])

array([[1, 1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0, 1]])

Dans l'exemple ci-dessus, l'analyseur `char_wb` est utilisé, ce qui crée des n-grammes uniquement à partir de caractères situés à l'intérieur des limites des mots (remplis d'espaces de chaque côté). L'analyseur `char`, en revanche, crée des n-grammes qui s'étendent à travers les mots :

In [2]:
ngram_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(5, 5))
ngram_vectorizer.fit_transform(['jumpy fox'])
# <1x4 sparse matrix of type '<... 'numpy.int64'>'
#    with 4 stored elements in Compressed Sparse ... format>
ngram_vectorizer.get_feature_names_out()
# array([' fox ', ' jump', 'jumpy', 'umpy '], ...)

ngram_vectorizer = CountVectorizer(analyzer='char', ngram_range=(5, 5))
ngram_vectorizer.fit_transform(['jumpy fox'])
# <1x5 sparse matrix of type '<... 'numpy.int64'>'
#     with 5 stored elements in Compressed Sparse ... format>

ngram_vectorizer.get_feature_names_out()
# array(['jumpy', 'mpy f', 'py fo', 'umpy ', 'y fox'], ...)

array(['jumpy', 'mpy f', 'py fo', 'umpy ', 'y fox'], dtype=object)

La variante `char_wb` qui prend en compte les limites des mots est particulièrement intéressante pour les langues qui utilisent des espaces pour séparer les mots car elle génère des caractéristiques beaucoup moins bruyantes que la variante brute `char` dans ce cas. Pour de telles langues, elle peut améliorer à la fois l'exactitude prédictive et la vitesse de convergence des classifieurs entraînés à l'aide de ces caractéristiques tout en conservant la robustesse en ce qui concerne les fautes d'orthographe et les dérivations de mots.

Bien que certaines informations de positionnement local puissent être préservées en extrayant des n-grammes au lieu de mots individuels, les sacs de mots et les sacs de n-grammes détruisent la plupart de la structure interne du document et donc la plupart de la signification portée par cette structure interne.

Pour aborder la tâche plus large de la compréhension du langage naturel, la structure locale des phrases et des paragraphes doit donc être prise en compte. Beaucoup de ces modèles seront donc conçus comme des problèmes de "sortie structurée", qui sont actuellement hors du champ d'application de scikit-learn.

## <a id='vectorizing-a-large-text-corpus-with-the-hashing-trick'></a> 6.2.3.8. Vectoriser un grand corpus de textes à l'aide de l'astuce du hachage

Le schéma de vectorisation ci-dessus est simple mais le fait qu'il contienne une **correspondance en mémoire des jetons de chaîne avec les indices de caractéristiques entiers** (l'attribut de `vocabulary_`) entraîne plusieurs **problèmes lors de la manipulation de grands ensembles de données** :
* plus le corpus est grand, plus le vocabulaire se développera et donc la consommation de mémoire également,
* l'ajustement nécessite l'allocation de structures de données intermédiaires de taille proportionnelle à celle de l'ensemble de données d'origine.
* la construction de la correspondance de mots nécessite un passage complet sur l'ensemble de données, il n'est donc pas possible d'adapter les classifieurs de texte de manière strictement en ligne.
* la conservation et la désérialisation de vectoriseurs avec un grand `vocabulary_` peut être très lente (typiquement beaucoup plus lente que la conservation / la désérialisation de structures de données plates comme un tableau NumPy de la même taille),
* il n'est pas facilement possible de diviser le travail de vectorisation en sous-tâches concurrentes car l'attribut de `vocabulary_` devrait être un état partagé avec une barrière de synchronisation à grains fins : la correspondance entre la chaîne de jeton et l'indice de caractéristique dépend de l'ordre de la première occurrence de chaque jeton et devrait donc être partagée, ce qui pourrait nuire aux performances des travailleurs concurrents au point de les rendre plus lents que la variante séquentielle.

Il est possible de surmonter ces limitations en combinant le "hashing trick" ([**Hachage de caractéristiques** (6.2.2)](https://scikit-learn.org/stable/modules/feature_extraction.html#feature-hashing)) implémenté par la classe [**`FeatureHasher`**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.FeatureHasher.html) et les fonctionnalités de prétraitement et de tokenization de [**`CountVectorizer`**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html).

Cette combinaison est mise en œuvre dans [**`HashingVectorizer`**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html), une classe de transformeur qui est en grande partie compatible avec l'API de [**`CountVectorizer`**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). [**`HashingVectorizer`**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html) est sans état, ce qui signifie que vous n'avez pas besoin de l'ajuster avec `fit` :

In [4]:
from sklearn.feature_extraction.text import HashingVectorizer
hv = HashingVectorizer(n_features=10)
hv.transform(corpus)
# <4x10 sparse matrix of type '<... 'numpy.float64'>'
#     with 16 stored elements in Compressed Sparse ... format>

NameError: name 'corpus' is not defined

Vous pouvez voir que 16 jetons de caractéristiques non nuls ont été extraits dans la sortie vectorielle : c'est moins que les 19 non nuls extraits précédemment par [**`CountVectorizer`**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) sur le même corpus jouet. La divergence vient des collisions de la fonction de hachage en raison de la faible valeur du paramètre `n_features`.

Dans un cadre réel, le paramètre `n_features` peut être laissé à sa valeur par défaut de `2 ** 20` (environ un million de caractéristiques possibles). Si la mémoire ou la taille des modèles en aval est un problème, la sélection d'une valeur inférieure, telle que `2 ** 18`, pourrait aider sans introduire trop de collisions supplémentaires sur les tâches de classification de texte typiques.

Notez que la dimensionnalité n'affecte pas le temps d'entraînement CPU des algorithmes qui fonctionnent sur des matrices CSR `(LinearSVC(dual=True)`, `Perceptron`, `SGDClassifier`, `PassiveAggressive`), mais elle affecte les algorithmes qui fonctionnent avec des matrices CSC `(LinearSVC(dual=False)`, `Lasso()`, etc.).

Essayons à nouveau avec les paramètres par défaut :

In [5]:
hv = HashingVectorizer()
hv.transform(corpus)

NameError: name 'corpus' is not defined

Nous n'obtenons plus de collisions, mais cela se fait au prix d'une dimensionnalité beaucoup plus grande de l'espace de sortie. Bien sûr, d'autres termes que les 19 utilisés ici peuvent encore entrer en collision les uns avec les autres.

Le [**`HashingVectorizer`**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html) présente également les limitations suivantes :
* il n'est pas possible d'inverser le modèle (pas de méthode `inverse_transform`), ni d'accéder à la représentation de chaîne d'origine des caractéristiques, en raison de la nature unidirectionnelle de la fonction de hachage qui effectue la correspondance.
* il ne fournit pas de pondération IDF car cela introduirait une dimension d'état dans le modèle. Un [**`TfidfTransformer`**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html) peut lui être ajouté dans un pipeline si nécessaire.

## <a id='performing-out-of-core-scaling-with-hashingvectorizer'></a> 6.2.3.9. Effectuer une mise à l'échelle hors ligne avec HashingVectorizer

Un développement intéressant de l'utilisation de [**`HashingVectorizer`**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html) est la capacité à effectuer une mise à l'échelle hors ligne. Cela signifie que nous pouvons apprendre à partir de données qui ne rentrent pas dans la mémoire principale de l'ordinateur.

Une stratégie pour mettre en œuvre une mise à l'échelle hors ligne consiste à diffuser des données à l'estimateur en mini-lots. Chaque mini-lot est vectorisé en utilisant [**`HashingVectorizer`**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html) afin de garantir que l'espace d'entrée de l'estimateur a toujours la même dimensionnalité. La quantité de mémoire utilisée à tout moment est donc limitée par la taille d'un mini-lot. Bien qu'il n'y ait pas de limite à la quantité de données pouvant être traitées en utilisant cette approche, d'un point de vue pratique, le temps d'apprentissage est souvent limité par le temps CPU que l'on veut consacrer à la tâche.

Pour un exemple complet de mise à l'échelle hors ligne dans une tâche de classification de texte, consultez l'exemple [**Classification hors ligne de documents textuels**](https://scikit-learn.org/stable/auto_examples/applications/plot_out_of_core_classification.html).

## <a id='customizing-the-vectorizer-classes'></a> 6.2.3.10. Personnalisation des classes vectorizer

Il est possible de personnaliser le comportement en passant un objet appelable au constructeur du vectoriseur.

In [6]:
def my_tokenizer(s):
    return s.split()

vectorizer = CountVectorizer(tokenizer=my_tokenizer)
vectorizer.build_analyzer()(u"Some... punctuation!") == (
    ['some...', 'punctuation!'])
# True

True

En particulier, nous nommons :
* `preprocessor` : une fonction qui prend un document entier en entrée (sous forme d'une chaîne de caractères unique) et renvoie une version éventuellement transformée du document, toujours sous forme d'une chaîne de caractères unique. Cela peut être utilisé pour supprimer les balises HTML, mettre tout le document en minuscules, etc.
* `tokenizer` : une fonction qui prend la sortie du preprocessor et la divise en jetons, puis renvoie une liste de ceux-ci.
* `analyzer` : une fonction qui remplace le preprocessor et le tokenizer. Les analyseurs par défaut appellent tous le preprocessor et le tokenizer, mais les analyseurs personnalisés peuvent sauter cette étape. L'extraction de n-grammes et le filtrage des mots vides ont lieu au niveau de l'analyseur, de sorte qu'un analyseur personnalisé peut devoir reproduire ces étapes.

(Les utilisateurs de Lucene pourraient reconnaître ces noms, mais il faut savoir que les concepts de scikit-learn ne se mappent pas nécessairement de manière univoque sur les concepts de Lucene.)

Pour rendre le preprocessor, le tokenizer et les analyseurs conscients des paramètres du modèle, il est possible de dériver de la classe et de remplacer les méthodes d'usine `build_preprocessor`, `build_tokenizer` et `build_analyzer` au lieu de passer des fonctions personnalisées.

Quelques astuces :
* Si les documents sont pré-découpés par un package externe, alors stockez-les dans des fichiers (ou des chaînes) avec les tokens séparés par des espaces et passez `analyzer=str.split`.
* Les analyses de jetons avancées telles que l'la racinisation, la lemmatisation, la séparation en composés, le filtrage en fonction de la partie du discours, etc. ne sont pas incluses dans le codebase de scikit-learn, mais peuvent être ajoutées en personnalisant soit le tokenizer, soit l'analyseur. Voici un `CountVectorizer` avec un tokenizer et un lemmatizer utilisant [**NLTK**](https://www.nltk.org/) :

In [7]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

vect = CountVectorizer(tokenizer=LemmaTokenizer())

(Notez que cela ne filtrera pas la ponctuation.)

L'exemple suivant transformera par exemple l'orthographe britannique en orthographe américaine :

In [8]:
import re
def to_british(tokens):
    for t in tokens:
        t = re.sub(r"(...)our$", r"\1or", t)
        t = re.sub(r"([bt])re$", r"\1er", t)
        t = re.sub(r"([iy])s(e$|ing|ation)", r"\1z\2", t)
        t = re.sub(r"ogue$", "og", t)
        yield t

class CustomVectorizer(CountVectorizer):
    def build_tokenizer(self):
        tokenize = super().build_tokenizer()
        return lambda doc: list(to_british(tokenize(doc)))

print(CustomVectorizer().build_analyzer()(u"color colour"))
# [...'color', ...'color']

['color', 'color']


pour d'autres styles de prétraitement; des exemples incluent la racinisation, la lemmatisation ou la normalisation des jetons numériques, cette dernière étant illustrée dans : [**Biclustering documents with the Spectral Co-clustering algorithm**](https://scikit-learn.org/stable/auto_examples/bicluster/plot_bicluster_newsgroups.html)

La personnalisation du vectoriseur peut également être utile lors du traitement de langues asiatiques qui n'utilisent pas de séparateur de mots explicite tel que l'espace.

# <a id='image-feature-extraction'></a> 6.2.4. Extraction de caractéristiques d'image

## <a id='patch-extraction'></a> 6.2.4.1. Extraction de patchs

La fonction [**`extract_patches_2d`**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.image.extract_patches_2d.html) extrait des patchs d'une image stockée sous forme de tableau bidimensionnel, ou tridimensionnel avec des informations de couleur le long du troisième axe. Pour reconstruire une image à partir de tous ses patchs, utilisez [**`reconstruct_from_patches_2d`**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.image.reconstruct_from_patches_2d.html). Par exemple, générons une image de 4x4 pixels avec 3 canaux de couleur (par exemple, en format RVB) :

In [9]:
import numpy as np
from sklearn.feature_extraction import image

one_image = np.arange(4 * 4 * 3).reshape((4, 4, 3))
one_image[:, :, 0]  # R channel of a fake RGB picture
# array([[ 0,  3,  6,  9],
#        [12, 15, 18, 21],
#        [24, 27, 30, 33],
#        [36, 39, 42, 45]])

patches = image.extract_patches_2d(one_image, (2, 2), max_patches=2,
    random_state=0)
patches.shape
# (2, 2, 2, 3)

patches[:, :, :, 0]
# array([[[ 0,  3],
#         [12, 15]],
# 
#        [[15, 18],
#         [27, 30]]])

patches = image.extract_patches_2d(one_image, (2, 2))
patches.shape
# (9, 2, 2, 3)

patches[4, :, :, 0]
# array([[15, 18],
#        [27, 30]])

array([[15, 18],
       [27, 30]])

Reconstruisons maintenant l'image d'origine à partir des patchs en effectuant une moyenne sur les zones de recouvrement :

In [10]:
reconstructed = image.reconstruct_from_patches_2d(patches, (4, 4, 3))
np.testing.assert_array_equal(one_image, reconstructed)

La classe [**`PatchExtractor`**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.image.PatchExtractor.html) fonctionne de la même manière que [**`extract_patches_2d`**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.image.extract_patches_2d.html), mais elle prend en charge plusieurs images en entrée. Elle est implémentée en tant qu'estimateur, elle peut donc être utilisée dans des pipelines. Voir :

In [11]:
five_images = np.arange(5 * 4 * 4 * 3).reshape(5, 4, 4, 3)
patches = image.PatchExtractor(patch_size=(2, 2)).transform(five_images)
patches.shape
# (45, 2, 2, 3)

(45, 2, 2, 3)

## <a id='connectivity-graph-of-an-image'></a> 6.2.4.2. Graphe de connectivité d'une image

Plusieurs estimateurs dans scikit-learn peuvent utiliser des informations de connectivité entre les caractéristiques ou les échantillons. Par exemple, la classification de Ward ([**classification hiérarchique** (2.3.6)](https://scikit-learn.org/stable/modules/clustering.html#hierarchical-clustering)) peut regrouper seulement les pixels voisins d'une image, formant ainsi des patches contigus :

<img alt="https://scikit-learn.org/stable/_images/sphx_glr_plot_coin_ward_segmentation_001.png" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_coin_ward_segmentation_001.png" style="width: 200.0px; height: 200.0px;" />

À cet effet, les estimateurs utilisent une matrice de "connectivité", indiquant quelles échantillons sont connectés.

La fonction [**`img_to_graph`**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.image.img_to_graph.html) renvoie une telle matrice à partir d'une image 2D ou 3D. De même, [**`grid_to_graph`**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.image.grid_to_graph.html) construit une matrice de connectivité pour les images étant donnée la forme de ces images.

Ces matrices peuvent être utilisées pour imposer la connectivité dans les estimateurs qui utilisent des informations de connectivité, tels que la classification de Ward (classification hiérarchique), mais aussi pour construire des noyaux précalculés, ou des matrices de similarité.

### Exemples

#### [**Une démo de clustering hiérarchique Ward structuré sur une image de pièces de monnaie**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_coin_ward_segmentation.ipynb)<br/>([*A demo of structured Ward hierarchical clustering on an image of coins*](https://scikit-learn.org/stable/auto_examples/cluster/plot_coin_ward_segmentation.html))

#### [**Classification spectrale pour la segmentation d'images**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_segmentation_toy.ipynb)<br/>([*Spectral clustering for image segmentation*](https://scikit-learn.org/stable/auto_examples/cluster/plot_segmentation_toy.html))

#### [**Agglomération de caractéristiques vs sélection univariée**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_feature_agglomeration_vs_univariate_selection.ipynb)<br/>([*Feature agglomeration vs. univariate selection*](https://scikit-learn.org/stable/auto_examples/cluster/plot_feature_agglomeration_vs_univariate_selection.html))